In [1]:
VERSION

v"1.8.5"

In [2]:
using SDDP,Test
using Pkg
using DataFrames,XLSX,CSV
using JSON
using Gurobi
using JuMP
import MathOptInterface as MOI
using Statistics
using DelimitedFiles
using Base

In [ ]:
##importing the Datasets and constants

cascade_df= DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "Cascade"))
name=cascade_df[:,11]
arc_df=DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "arc_cap",header=false))
max_df=DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "max_pwr",header=false))
a_df=DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "A",header=false))
a1_df=DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "A1",header=false))
a2_df=DataFrame(XLSX.readtable("input/Cascade_info.xlsx", "A2",header=false))
initial_storage_factor=[1,1,.99,.98,.85,.99]/10^6 
min_gen_factor=1
constant=1000
input= CSV.read("input/Initial_conditions.csv", DataFrame)
user_data=CSV.read("input/user_data.csv", DataFrame)
n_small_tree=user_data[6,2]
for i =1:nrow(cascade_df)
    if cascade_df[i,4]<input[3,i+1]/constant
        cascade_df[i,5]=input[3,i+1]/constant
    end
end
cascade_df=coalesce.(cascade_df,NaN)
# make a list for max decrease and min decrease
max_dec_list=[]
max_inc_list=[]
for i =1:nrow(cascade_df)
    if !isnan(cascade_df[i,8])
        push!(max_dec_list,i)
    end
    if !isnan(cascade_df[i,9])
        push!(max_inc_list,i)
    end
end
graph = SDDP.Graph(:0)
for z =0:n_small_tree
    shifted="$z"
    print("lets start $z shifted")
    df = DataFrame(XLSX.readtable("input/tree_2week_shifted_$z.xlsx", "Sheet1"))
    first(df,7)
    n_stages=maximum(df.Stage)
    graph = SDDP.Graph(0)
    for i in 1:length(df.Node)
        SDDP.add_node(graph, i)
        SDDP.add_edge(graph, df.Parent[i] => df.Node[i], Float64(df.Prob[i]))
    end
    graph
    struct Reservoir
        min_s::Float64              ## minimum storage volume(af)
        max_s::Float64              ## maximum storage volume(af)
        storage_initial::Float64    ## initial storage volume(af)
                                    ##!!!IMPORTANT: change each month
        arc_cap::Vector{Float64}    ## arc capacity (cfs)
        max_pwr::Vector{Float64}    ## max power generation (kw)
        spill_cap::Float64          ## spill capacity (cfs)
        release_min::Float64        ## min release (cfs) 
        release_initial::Float64    ## initial release (cfs) (from 50 year historical records 1970-2019)
        k::Float64                  ## k coefficient  (kw/cfs)     avg (energy(MWH)*1000/24) / avg(release)        
        n_unit::Int64               ## number of generators 
        max_decrease::Float64       ## maximum daily descrease rate of release change (cfs)
        max_increase::Float64       ## maximum daily inscrease rate of release change (cfs)
        min_gen::Float64            ## daily mean generation follow USACE forecast information (MW) (added 8/11/2022)
                                    ##!!!IMPORTANT: change each month
        evap_avg_monthly::Float64   ## monthly avg evaporation (1000 af)
    end
    #
 ## Define System connectivity matrix and Engineering data
A=Matrix{Float64}(a_df)

    ## Decomposed matrices for today and yesterday release
A1=Matrix{Float64}(a1_df)
A2=Matrix{Float64}(a2_df)
    ## Define Cascade system engineering data, include 6 reservoirs, each reservoir have the format Reservoir(min_s, max_s, initial,....) following
    #  the reservoir data structure defined in the last code cell.

Cascade_system=Reservoir[]
for i =1:nrow(arc_df)
    arc=[]
    pwr=[]
    for j =1:ncol(arc_df)
        if arc_df[i,j]>0
            push!(arc,arc_df[i,j])
            push!(pwr,max_df[i,j])
        end
    end
    temp=Reservoir(cascade_df[i,1],cascade_df[i,2],input[1,i+1]*initial_storage_factor[i],arc, 
        pwr,cascade_df[i,3],cascade_df[i,4],cascade_df[i,5], cascade_df[i,6], 
        cascade_df[i,7], cascade_df[i,8],cascade_df[i,9],input[2,i+1],cascade_df[i,10])
    push!(Cascade_system,temp)
end
###########################
 ## Define the model structure, constraints, and objective ##
    model = SDDP.PolicyGraph(
        graph,
        sense = :Min,
        lower_bound = 0,
       optimizer = Gurobi.Optimizer,
#        optimizer= MOI.OptimizerWithAttributes(Gurobi.Optimizer,
#         "Presolve"=>0,
#          "ScaleFlag"=>3) 
         )do subproblem, node
    @variables(subproblem, begin
        Cascade_system[r].min_s <= storage[r=1:nrow(cascade_df)] <= Cascade_system[r].max_s, SDDP.State,( initial_value = Cascade_system[r].storage_initial)
        release[r=1:nrow(cascade_df)] >= Cascade_system[r].release_min, SDDP.State,(initial_value = Cascade_system[r].release_initial)
        0 <= gen_output[r =1:nrow(cascade_df),u=1:Cascade_system[r].n_unit] <= Cascade_system[r].max_pwr[u]
        0 <= power_flow[r =1:nrow(cascade_df),u=1:Cascade_system[r].n_unit] <= Cascade_system[r].arc_cap[u]
        inflow[r=1:nrow(cascade_df)] == df[!,r+4][node]
        totaldemand == df[!,end-1][node]
        0 <= spill[r=1:nrow(cascade_df)] <= Cascade_system[r].spill_cap
        total_gen[r=1:nrow(cascade_df)] >= Cascade_system[r].min_gen/constant
        total_pflow[r=1:nrow(cascade_df)] >=0
        end)
    @constraints(subproblem,begin
        sum(total_gen[r]  for r=1:nrow(cascade_df)) == totaldemand
        [r=1:nrow(cascade_df),u=1:Cascade_system[r].n_unit],gen_output[r,u]  ==  power_flow[r,u]* Cascade_system[r].k
        [r=1:nrow(cascade_df)], total_gen[r]*1000 == sum(gen_output[r,u] for u=1:Cascade_system[r].n_unit)
        [r=1:nrow(cascade_df)],  total_pflow[r] == sum(power_flow[r,u] for u=1:Cascade_system[r].n_unit)
        [r=1:nrow(cascade_df)],release[r].out == total_pflow[r] + spill[r]
[r=max_dec_list],-Cascade_system[r].max_decrease <= release[r].out - release[r].in
        [r=max_inc_list],  Cascade_system[r].max_increase >= release[r].out - release[r].in
        [r = 1:nrow(cascade_df)], storage[r].out == storage[r].in + (df[node, Symbol("Inflow$r")] + sum(release[j].out*A[r,j] for j=1:r))*1.9835/constant   
        end)
    @stageobjective(subproblem, sum(spill[r] for r=1:nrow(cascade_df)))
end
    #################
    ## Training the model, and obtaining the optimal policy
    print("lets train $z shifted\n")
    SDDP.train(model; iteration_limit = 1000)
    print("lets sim $z shifted\n")
    simulations = SDDP.simulate(
        # The trained model to simulate.
        model,
        # The number of replications.
        # A list of names to record the values of.
        1000,
        [:storage, :inflow,:totaldemand,:power_flow,:gen_output,:spill,:total_gen,:total_pflow,:release]
    )
    ###################
    #Extract the quantile value for ribbon figures, example for GP release
    #for total_pflow
    for r=1:nrow(cascade_df)
      quan=[]
      for stage =1:n_stages    #loop over stage
        d=[]
        for i=1:1000      #loop over simulations
          temp1 =simulations[:][i][stage][:total_pflow][r]
          push!(d,temp1) 
        end
        temp2 = quantile(d,[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
        push!(quan,temp2)
      end
      col1=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
      table=[[col1];quan]
      df1=DataFrame(table,:auto)
      fname= name[r] * "_total_pflow_$shifted.xlsx"
      full_name = joinpath("output/quantile_files/shifted", fname)
      XLSX.writetable(full_name, collect(eachcol(df1)), names(df1))
    end
    #for storage
    for r=1:nrow(cascade_df)
      quan=[]
      for stage =1:n_stages    #loop over stage
        d=[]
        for i=1:1000      #loop over simulations
          temp1 =simulations[:][i][stage][:storage][r].out
          push!(d,temp1) 
        end
        temp2 = quantile(d,[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
        push!(quan,temp2)
      end
      col1=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
      table=[[col1];quan]
      df1=DataFrame(table,:auto)
      fname= name[r] * "_storage_$shifted.xlsx" 
      full_name = joinpath("output/quantile_files/shifted", fname)
      XLSX.writetable(full_name, collect(eachcol(df1)), names(df1))
    end
    #for release
    for r=1:nrow(cascade_df)
      quan=[]
      for stage =1:n_stages    #loop over stage
        d=[]
        for i=1:1000      #loop over simulations
          temp1 =simulations[:][i][stage][:release][r].out
          push!(d,temp1) 
        end
        temp2 = quantile(d,[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
        push!(quan,temp2)
      end
      col1=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
      table=[[col1];quan]
      df1=DataFrame(table,:auto)
      fname= name[r] * "_release_$shifted.xlsx" 
      full_name = joinpath("output/quantile_files/shifted", fname)
      XLSX.writetable(full_name, collect(eachcol(df1)), names(df1))
    end
    #for spill
    for r=1:nrow(cascade_df)
      quan=[]
      for stage =1:n_stages    #loop over stage
        d=[]
        for i=1:1000      #loop over simulations
          temp1 =simulations[:][i][stage][:spill][r]
          push!(d,temp1) 
        end
        temp2 = quantile(d,[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0])
        push!(quan,temp2)
      end
      col1=[0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
      table=[[col1];quan]
      df1=DataFrame(table,:auto)
      fname= name[r] * "_spill_$shifted.xlsx" 
      full_name = joinpath("output/quantile_files/shifted", fname)
      XLSX.writetable(full_name, collect(eachcol(df1)), names(df1))
    end    
    ###################
    ###Solution extraction
##This extract the optimal solution for all the nodes
    #spill
    spill_solution=[]
    a = Dict(i => [simulations[i][j][:node_index] for j =1:n_stages] for i =1:1000) 
    b = Dict(df.Node[i] => df.Stage[i] for i =1:length(df.Node))
    for node =1:length(df.Node)
        c=[]
        for i in keys(a)
            if node in a[i]
                push!(c,i)
            end 
        end
        if c ==[]
            println(0)
        else
            temp=simulations[c[1]][b[node]][:spill]
            push!(spill_solution,temp)
        end
    end
    writedlm("output/temp_files/node_solutions/shifted_tree/Nodes_spill_solution_shifted_$shifted.csv", Tables.table(spill_solution),'\n')
########
#release
    release_solution=[]
    a = Dict(i => [simulations[i][j][:node_index] for j =1:n_stages] for i =1:1000) 
    b = Dict(df.Node[i] => df.Stage[i] for i =1:length(df.Node))
    for node =1:length(df.Node)
        c=[]
        for i in keys(a)
            if node in a[i]
                push!(c,i)
            end 
        end
        if c ==[]
            println(0)
        else
            temp=simulations[c[1]][b[node]][:release]
            push!(release_solution,temp)
        end
    end
    writedlm("output/temp_files/node_solutions/shifted_tree/Nodes_release_solution_shifted_$shifted.csv", Tables.table(release_solution),'\n')
##########
    #storage
    storage_solution=[]
    a = Dict(i => [simulations[i][j][:node_index] for j =1:n_stages] for i =1:1000) 
    b = Dict(df.Node[i] => df.Stage[i] for i =1:length(df.Node))
    for node =1:length(df.Node)
        c=[]
        for i in keys(a)
            if node in a[i]
                push!(c,i)
            end 
        end
        if c ==[]
            println(0)
        else
            temp=simulations[c[1]][b[node]][:storage]
            push!(storage_solution,temp)
        end
    end
    writedlm("output/temp_files/node_solutions/shifted_tree/Nodes_storage_solution_shifted_$shifted.csv", Tables.table(storage_solution),'\n')
####
    #total_pflow
    total_pflow_solution=[]
    a = Dict(i => [simulations[i][j][:node_index] for j =1:n_stages] for i =1:1000) 
    b = Dict(df.Node[i] => df.Stage[i] for i =1:length(df.Node))
    for node =1:length(df.Node)
        c=[]
        for i in keys(a)
            if node in a[i]
                push!(c,i)
            end 
        end
        if c ==[]
            println(0)
        else
            temp=simulations[c[1]][b[node]][:total_pflow]
            push!(total_pflow_solution,temp)
        end
    end
    writedlm("output/temp_files/node_solutions/shifted_tree/Nodes_total_pflow_solution_shifted_$shifted.csv", Tables.table(total_pflow_solution),'\n')
    ##############end of for##########    
    print("results exctracted for $z shifted\n")
end

lets start 0 shiftedlets train 0 shifted
------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-21

Problem
  Nodes           : 211
  State variables : 12
  Scenarios       : 3.00000e+01
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (122, 122)
    VariableRef in MOI.EqualTo{Float64}     : (7, 7)
    VariableRef in MOI.LessThan{Float64}    : (84, 85)
    AffExpr in MOI.LessThan{Float64}        : (3, 3)
    AffExpr in MOI.GreaterThan{Float64}     : (2, 2)
    AffExpr in MOI.EqualTo{Float64}         : (61, 61)
    VariableRef in MOI.GreaterThan{Float64} : (103, 103)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [2e-03, 1e+03]
  Non-zero Objective range  [1e+00, 1e+00]
  Non-zero Bounds range     [7e-0

        1    6.556716e-02   0.000000e+00   1.321300e+01          1         35
        2    3.195591e+00   0.000000e+00   1.352600e+01          1         74
        3    0.000000e+00   0.000000e+00   1.355800e+01          1        106
        4    0.000000e+00   0.000000e+00   1.357800e+01          1        142
        5    0.000000e+00   0.000000e+00   1.359400e+01          1        175
        6    0.000000e+00   0.000000e+00   1.361400e+01          1        207
        7    0.000000e+00   0.000000e+00   1.363000e+01          1        240
        8    8.566033e+00   0.000000e+00   1.365800e+01          1        273
        9    0.000000e+00   0.000000e+00   1.366600e+01          1        312
       10    0.000000e+00   0.000000e+00   1.367800e+01          1        351
       11    0.000000e+00   0.000000e+00   1.368600e+01          1        383
       12    0.000000e+00   0.000000e+00   1.369400e+01          1        422
       13    0.000000e+00   0.000000e+00   1.371000e+01         

      110    0.000000e+00   0.000000e+00   1.489900e+01          1       3812
      111    0.000000e+00   0.000000e+00   1.491100e+01          1       3848
      112    0.000000e+00   0.000000e+00   1.492300e+01          1       3887
      113    0.000000e+00   0.000000e+00   1.493500e+01          1       3920
      114    0.000000e+00   0.000000e+00   1.494700e+01          1       3952
      115    0.000000e+00   0.000000e+00   1.495900e+01          1       3991
      116    0.000000e+00   0.000000e+00   1.497100e+01          1       4025
      117    0.000000e+00   0.000000e+00   1.498300e+01          1       4064
      118    0.000000e+00   0.000000e+00   1.499900e+01          1       4103
      119    0.000000e+00   0.000000e+00   1.500700e+01          1       4135
      120    0.000000e+00   0.000000e+00   1.502300e+01          1       4167
      121    0.000000e+00   0.000000e+00   1.503500e+01          1       4206
      122    0.000000e+00   0.000000e+00   1.505500e+01         

      216    0.000000e+00   0.000000e+00   1.647200e+01          1       7589
      217    0.000000e+00   0.000000e+00   1.649000e+01          1       7626
      218    0.000000e+00   0.000000e+00   1.650400e+01          1       7660
      219    0.000000e+00   0.000000e+00   1.652800e+01          1       7699
      220    0.000000e+00   0.000000e+00   1.654000e+01          1       7735
      221    0.000000e+00   0.000000e+00   1.655600e+01          1       7774
      222    0.000000e+00   0.000000e+00   1.657600e+01          1       7813
      223    0.000000e+00   0.000000e+00   1.659200e+01          1       7845
      224    0.000000e+00   0.000000e+00   1.661200e+01          1       7884
      225    0.000000e+00   0.000000e+00   1.662400e+01          1       7916
      226    0.000000e+00   0.000000e+00   1.664000e+01          1       7948
      227    0.000000e+00   0.000000e+00   1.665600e+01          1       7980
      228    0.000000e+00   0.000000e+00   1.667600e+01         

      324    0.000000e+00   0.000000e+00   1.853600e+01          1      11410
      325    0.000000e+00   0.000000e+00   1.856100e+01          1      11444
      326    0.000000e+00   0.000000e+00   1.858100e+01          1      11483
      327    0.000000e+00   0.000000e+00   1.865200e+01          1      11522
      328    0.000000e+00   0.000000e+00   1.866800e+01          1      11554
      329    0.000000e+00   0.000000e+00   1.868800e+01          1      11586
      330    0.000000e+00   0.000000e+00   1.870400e+01          1      11618
      331    0.000000e+00   0.000000e+00   1.872400e+01          1      11653
      332    0.000000e+00   0.000000e+00   1.874000e+01          1      11687
      333    0.000000e+00   0.000000e+00   1.876400e+01          1      11726
      334    0.000000e+00   0.000000e+00   1.878400e+01          1      11758
      335    0.000000e+00   0.000000e+00   1.881300e+01          1      11797
      336    0.000000e+00   0.000000e+00   1.882900e+01         

...

      218    0.000000e+00   0.000000e+00   3.150000e+00          1       7624
      219    0.000000e+00   0.000000e+00   3.220000e+00          1       7655
      220    0.000000e+00   0.000000e+00   3.240000e+00          1       7686
      221    0.000000e+00   0.000000e+00   3.250000e+00          1       7721
      222    0.000000e+00   0.000000e+00   3.272000e+00          1       7760
      223    0.000000e+00   0.000000e+00   3.290000e+00          1       7795
      224    0.000000e+00   0.000000e+00   3.300000e+00          1       7830
      225    0.000000e+00   0.000000e+00   3.320000e+00          1       7865
      226    0.000000e+00   0.000000e+00   3.330000e+00          1       7900
      227    0.000000e+00   0.000000e+00   3.355000e+00          1       7936
      228    0.000000e+00   0.000000e+00   3.372000e+00          1       7971
      229    0.000000e+00   0.000000e+00   3.380000e+00          1       8006
      230    0.000000e+00   0.000000e+00   3.400000e+00         

      326    0.000000e+00   0.000000e+00   5.140000e+00          1      11372
      327    0.000000e+00   0.000000e+00   5.160000e+00          1      11411
      328    0.000000e+00   0.000000e+00   5.180000e+00          1      11447
      329    0.000000e+00   0.000000e+00   5.200000e+00          1      11483
      330    0.000000e+00   0.000000e+00   5.210000e+00          1      11517
      331    0.000000e+00   0.000000e+00   5.230000e+00          1      11550
      332    0.000000e+00   0.000000e+00   5.250000e+00          1      11587
      333    0.000000e+00   0.000000e+00   5.274000e+00          1      11628
      334    0.000000e+00   0.000000e+00   5.290000e+00          1      11663
      335    0.000000e+00   0.000000e+00   5.310000e+00          1      11698
      336    0.000000e+00   0.000000e+00   5.330000e+00          1      11729
      337    0.000000e+00   0.000000e+00   5.350000e+00          1      11764
      338    0.000000e+00   0.000000e+00   5.374000e+00         

      433    0.000000e+00   0.000000e+00   7.450000e+00          1      15111
      434    0.000000e+00   0.000000e+00   7.470000e+00          1      15147
      435    0.000000e+00   0.000000e+00   7.490000e+00          1      15187
      436    0.000000e+00   0.000000e+00   7.520000e+00          1      15218
      437    0.000000e+00   0.000000e+00   7.540000e+00          1      15252
      438    0.000000e+00   0.000000e+00   7.565000e+00          1      15285
      439    0.000000e+00   0.000000e+00   7.595000e+00          1      15316
      440    0.000000e+00   0.000000e+00   7.615000e+00          1      15351
      441    0.000000e+00   0.000000e+00   7.640000e+00          1      15382
      442    0.000000e+00   0.000000e+00   7.665000e+00          1      15417
      443    0.000000e+00   0.000000e+00   7.695000e+00          1      15450
      444    0.000000e+00   0.000000e+00   7.715000e+00          1      15486
      445    0.000000e+00   0.000000e+00   7.740000e+00         

Excessive output truncated after 524289 bytes.

      476    0.000000e+00   0.000000e+00   8.570000e+00          1      16625
      477    0.000000e+00   0.000000e+00   8.590000e+00          1      16660
      478    0.000000e+00   0.000000e+00

In [ ]:
cascade_df

In [ ]:
print("Code C_2 is Done! Now run Code D")